<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# `GiRaFFE_NRPy_staggered`: A-to-B code

## Author: Patrick Nelson

**Notebook Status:** <font color='green'><b>Validated</b></font>

**Validation Notes:** This code is a port from the old `GiRaFFE`

### NRPy+ Source Code for this module:
* [GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_A2B.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_A2B.py)

## Introduction:

This notebook presents an alternate algorithm for computing the magnetic field from the vector potential in a staggered prescription. It is a direct port of the old `GiRaFFE` implementation.

We will need to compute $B^i$ everywhere in order to evolve $\tilde{S}_i$. However, $B^i = \epsilon^{ijk} \partial_j A_k$ requires derivatives of $A_i$, so getting $B^i$ in the ghostzones (and not just on the interior) will require some additional work. In the staggered prescription, this is accomplished with a simple copy boundary condition at the ghost zones where $i$, $j$, or $k=0$. Due to the staggered nature of the grids, the other ghost zone requires no such treatment. 

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

0. [Step 1](#prelim): Preliminaries
1. [Step 2](#code): Write the C code
    1. [Step 2.a](#simple_loop): Define a simple loop macro
    1. [Step 2.b](#function_definition): Write the function definition
    1. [Step 2.c](#metric_determinant): Find the metric determinant
    1. [Step 2.d](#indices): Set indices and apply copy boundary conditions
    1. [Step 2.e](#staggered): Compute the staggered magnetic field
    1. [Step 2.f](#centered): Compute the centered magnetic field
1. [Step 3](#code_validation): Code Validation
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='prelim'></a>

# Step 1: Preliminaries \[Back to [top](#toc)\]
$$\label{prelim}$$

This first block of code just sets up a subdirectory within `GiRaFFE_standalone_Ccodes/` to which we will write the C code.

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface
Ccodesdir = "GiRaFFE_standalone_Ccodes/A2B"
cmd.mkdir(os.path.join(Ccodesdir))

<a id='code'></a>

# Step 2: Write the C code \[Back to [top](#toc)\]
$$\label{code}$$

<a id='simple_loop'></a>

## Step 2.a: Define a simple loop macro \[Back to [top](#toc)\]
$$\label{simple_loop}$$

We first define a simple loop macro that loops over the entire grid.

In [2]:
%%writefile $Ccodesdir/compute_B_and_Bstagger_from_A.h
#define LOOP_DEFINE_SIMPLE                      \
  _Pragma("omp parallel for")                   \
  for(int k=0;k<Nxx_plus_2NGHOSTS2;k++)                \
    for(int j=0;j<Nxx_plus_2NGHOSTS1;j++)              \
      for(int i=0;i<Nxx_plus_2NGHOSTS0;i++)


Overwriting GiRaFFE_standalone_Ccodes/A2B/compute_B_and_Bstagger_from_A.h


<a id='function_definition'></a>

## Step 2.b: Write the function definition \[Back to [top](#toc)\]
$$\label{function_definition}$$

Here, we give our function definition. We pass our parameter construct, which contains needed quantities like the grid parameters, as well as the three metric, since we need its determinant for the curl in curved spacetime. Naturally, we also pass the vector potential (which we will differentiate) and the magnetic field (our output). Note that we are storing two different versions of the magnetic field. `Bx`, `By`, and `Bz` sample the magnetic field at cell centers, while `Bx_stagger`, `By_stagger`, and `Bz_stagger` sample the field at cell faces (e.g., $B^x$ is stored at $i+1/2, j, k$).

Finally, we also declare variables from the paramstruct in the normal way with the `#include` directive. (If you're reading this code and can't figure out where a variable is set, check here!)

In [3]:
%%writefile -a $Ccodesdir/compute_B_and_Bstagger_from_A.h

void GiRaFFE_compute_B_and_Bstagger_from_A(const paramstruct *params,
                                           const REAL *gxx, const REAL *gxy, const REAL *gxz, const REAL *gyy, const REAL *gyz,const REAL *gzz,
                                           REAL *psi3_bssn, const REAL *Ax, const REAL *Ay, const REAL *Az,
                                           REAL *Bx, REAL *By, REAL *Bz, REAL *Bx_stagger, REAL *By_stagger, REAL *Bz_stagger) {
#include "../set_Cparameters.h"


Appending to GiRaFFE_standalone_Ccodes/A2B/compute_B_and_Bstagger_from_A.h


<a id='metric_determinant'></a>

## Step 2.c: Find the metric determinant \[Back to [top](#toc)\]
$$\label{metric_determinant}$$

In this first loop, we will compute the fourth root of the metric determinant at the cell centers, and temporarily store it in `psi3_bssn`. We do this because, while we will ultimately want the square root of the metric determinant, we will use the geometric mean to compute its value between grid points. (The geometric mean of $x$ and $y$ is $\sqrt{xy}$.)

In [4]:
%%writefile -a $Ccodesdir/compute_B_and_Bstagger_from_A.h

  LOOP_DEFINE_SIMPLE {
    const int index=IDX3S(i,j,k);
    psi3_bssn[index] = sqrt(sqrt( gxx[index]*gyy[index]*gzz[index]
                               -  gxx[index]*gyz[index]*gyz[index]
                               +2*gxy[index]*gxz[index]*gyz[index]
                               -  gyy[index]*gxz[index]*gxz[index]
                               -  gzz[index]*gxy[index]*gxy[index]));
  }


Appending to GiRaFFE_standalone_Ccodes/A2B/compute_B_and_Bstagger_from_A.h


<a id='indices'></a>

## Step 2.d: Set indices and apply copy boundary conditions \[Back to [top](#toc)\]
$$\label{indices}$$

Next, we will define the indices that we care about for a given point $i,j,k$ where we will find the magnetic field. For the derivatives we're taking, these happen to be $i$, $i-1$, $j$, $j-1$, $k$, and $k-1$. Here, we also implement our copy boundary condition. Note that in C, `(i!=0)` will evaluate to `0` only if `i = 0`, and `1` otherwise. So, for any nonzero `i`, $i$ and $i+1$ will be exactly what one would expect. However, for `i=0`, we will store `shiftedim1=0` and `shiftedi=1` just as we do for `i=1` (hence 'copy'), preventing us from trying to access out-of-bounds memory, which would be bad. (Note that our loop bounds will prevent us from running into other such out-of-bounds problems, and that all of this applies exactly the same for $j$ and $k$.)

We will also declare variables to store the indices we will need later; `actual_index` is where we will store the calculated magnetic field. We will set it now, since it will not change. We also invert $\psi^3 = \sqrt{\sqrt{\gamma}}$.

In [5]:
%%writefile -a $Ccodesdir/compute_B_and_Bstagger_from_A.h

  LOOP_DEFINE_SIMPLE {
    // Look Mom, no if() statements!
    const int shiftedim1 = (i-1)*(i!=0); // This way, i=0 yields shiftedim1=0 and shiftedi=1, used below for our COPY boundary condition.
    const int shiftedi   = shiftedim1+1;

    const int shiftedjm1 = (j-1)*(j!=0);
    const int shiftedj   = shiftedjm1+1;

    const int shiftedkm1 = (k-1)*(k!=0);
    const int shiftedk   = shiftedkm1+1;

    int index,indexim1,indexjm1,indexkm1;

    const int actual_index = IDX3S(i,j,k);

    const REAL Psim3 = 1.0/psi3_bssn[actual_index];


Appending to GiRaFFE_standalone_Ccodes/A2B/compute_B_and_Bstagger_from_A.h


<a id='staggered'></a>

## Step 2.e: Compute the staggered magnetic field \[Back to [top](#toc)\]
$$\label{staggered}$$

We will compute the staggered magnetic field first, starting with $B^x = \partial_y A_z - \partial_z A_y$. Now,
* `Ax(i,j,k)` is actually $A_x(i,j+1/2,k+1/2)$,
* `Ay(i,j,k)` is actually $A_y(i+1/2,j,k+1/2)$, and 
* `Az(i,j,k)` is actually $A_z(i+1/2,j+1/2,k)$.

Likewise, 
* `Bx(i,j,k)` is actually $B^x(i+1/2,j,k)$,
* `By(i,j,k)` is actually $B^y(i,j+1/2,k)$, and 
* `Bz(i,j,k)` is actually $B^z(i,j,k+1/2)$.

So, $\partial_z A_y\ {\rm at}\ (i+1/2,j,k)$ is actually `Ay(i,j,k) - Ay(i,j,k-1)]/dZ`, and so forth. 

In [6]:
%%writefile -a $Ccodesdir/compute_B_and_Bstagger_from_A.h

    // For the lower boundaries, the following applies a "copy"
    //    boundary condition on Bi_stagger where needed.
    //    E.g., Bx_stagger(i,jmin,k) = Bx_stagger(i,jmin+1,k)
    //    We find the copy BC works better than extrapolation.
    // For the upper boundaries, we do the following copy:
    //    E.g., Psi(imax+1,j,k)=Psi(imax,j,k)
    /**************/
    /* Bx_stagger */
    /**************/

    index    = IDX3S(i,shiftedj,shiftedk);
    indexjm1 = IDX3S(i,shiftedjm1,shiftedk);
    indexkm1 = IDX3S(i,shiftedj,shiftedkm1);
    // Set Bx_stagger = \partial_y A_z - partial_z A_y
    // "Grid" Ax(i,j,k) is actually Ax(i,j+1/2,k+1/2)
    // "Grid" Ay(i,j,k) is actually Ay(i+1/2,j,k+1/2)
    // "Grid" Az(i,j,k) is actually Ay(i+1/2,j+1/2,k)
    // Therefore, the 2nd order derivative \partial_z A_y at (i+1/2,j,k) is:
    //          ["Grid" Ay(i,j,k) - "Grid" Ay(i,j,k-1)]/dZ
    Bx_stagger[actual_index] = (Az[index]-Az[indexjm1])*invdx1 - (Ay[index]-Ay[indexkm1])*invdx2;


Appending to GiRaFFE_standalone_Ccodes/A2B/compute_B_and_Bstagger_from_A.h


Now, we must divide by $\psi^6 = \sqrt{\gamma}$. However, this quantity has been calculated at cell centers; we will find it on the cell face by taking the geometric mean. For instance, for $B^x$, this involves the points at $i$ and $i+1$. We again use a copy boundary condition, but on the faces where $i,j,k$ take their maximum value. If `i+1` exceeds the allowed values, it is set to the maximum allowed value (due to our loop bounds, this is only ever a distance of one point).

In [7]:
%%writefile -a $Ccodesdir/compute_B_and_Bstagger_from_A.h

    // Now multiply Bx_stagger by 1/sqrt(gamma(i+1/2,j,k)]) = 1/sqrt(1/2 [gamma + gamma_ip1]) = exp(-6 x 1/2 [phi + phi_ip1] )
    const int imax_minus_i = (Nxx_plus_2NGHOSTS0-1)-i;
    const int indexip1jk = IDX3S(i + ( (imax_minus_i > 0) - (0 > imax_minus_i) ),j,k);
    Bx_stagger[actual_index] *= Psim3/psi3_bssn[indexip1jk];


Appending to GiRaFFE_standalone_Ccodes/A2B/compute_B_and_Bstagger_from_A.h


We repeat the above steps, except we now are setting $B^y = \partial_z A_x - \partial_x A_z$

In [8]:
%%writefile -a $Ccodesdir/compute_B_and_Bstagger_from_A.h

    /**************/
    /* By_stagger */
    /**************/

    index    = IDX3S(shiftedi,j,shiftedk);
    indexim1 = IDX3S(shiftedim1,j,shiftedk);
    indexkm1 = IDX3S(shiftedi,j,shiftedkm1);
    // Set By_stagger = \partial_z A_x - \partial_x A_z
    By_stagger[actual_index] = (Ax[index]-Ax[indexkm1])*invdx2 - (Az[index]-Az[indexim1])*invdx0;

    // Now multiply By_stagger by 1/sqrt(gamma(i,j+1/2,k)]) = 1/sqrt(1/2 [gamma + gamma_jp1]) = exp(-6 x 1/2 [phi + phi_jp1] )
    const int jmax_minus_j = (Nxx_plus_2NGHOSTS1-1)-j;
    const int indexijp1k = IDX3S(i,j + ( (jmax_minus_j > 0) - (0 > jmax_minus_j) ),k);
    By_stagger[actual_index] *= Psim3/psi3_bssn[indexijp1k];


Appending to GiRaFFE_standalone_Ccodes/A2B/compute_B_and_Bstagger_from_A.h


We repeat the above steps, except we now are setting $B^z = \partial_x A_y - \partial_y A_x$

In [9]:
%%writefile -a $Ccodesdir/compute_B_and_Bstagger_from_A.h

    /**************/
    /* Bz_stagger */
    /**************/

    index    = IDX3S(shiftedi,shiftedj,k);
    indexim1 = IDX3S(shiftedim1,shiftedj,k);
    indexjm1 = IDX3S(shiftedi,shiftedjm1,k);
    // Set Bz_stagger = \partial_x A_y - \partial_y A_x
    Bz_stagger[actual_index] = (Ay[index]-Ay[indexim1])*invdx0 - (Ax[index]-Ax[indexjm1])*invdx1;

    // Now multiply Bz_stagger by 1/sqrt(gamma(i,j,k+1/2)]) = 1/sqrt(1/2 [gamma + gamma_kp1]) = exp(-6 x 1/2 [phi + phi_kp1] )
    const int kmax_minus_k = (Nxx_plus_2NGHOSTS2-1)-k;
    const int indexijkp1 = IDX3S(i,j,k + ( (kmax_minus_k > 0) - (0 > kmax_minus_k) ));
    Bz_stagger[actual_index] *= Psim3/psi3_bssn[indexijkp1];

  }


Appending to GiRaFFE_standalone_Ccodes/A2B/compute_B_and_Bstagger_from_A.h


<a id='centered'></a>

## Step 2.f: Compute the centered magnetic field \[Back to [top](#toc)\]
$$\label{centered}$$

We now address the magnetic field as sampled at cell centers. We start in much the same way as above: we start our simple loop over all points, and declare our $i,j,k$ and $i-1$, $j-1$, and $k-1$ as before, with the same copy boundary condition.

In [10]:
%%writefile -a $Ccodesdir/compute_B_and_Bstagger_from_A.h

  LOOP_DEFINE_SIMPLE {
    // Look Mom, no if() statements!
    const int shiftedim1 = (i-1)*(i!=0); // This way, i=0 yields shiftedim1=0 and shiftedi=1, used below for our COPY boundary condition.
    const int shiftedi   = shiftedim1+1;

    const int shiftedjm1 = (j-1)*(j!=0);
    const int shiftedj   = shiftedjm1+1;

    const int shiftedkm1 = (k-1)*(k!=0);
    const int shiftedk   = shiftedkm1+1;

    int index,indexim1,indexjm1,indexkm1;

    const int actual_index = IDX3S(i,j,k);


Appending to GiRaFFE_standalone_Ccodes/A2B/compute_B_and_Bstagger_from_A.h


We now perform a basic interpolation (the most basic--a simple arithmetic mean of two quantities) to find the cell-centered magnetic field. Taking $B^x$ as an example, which has been calculated at $(i+1/2,j,k)$ for grid values `i,j,k`, we find its value at the cell center by averaging the values at $(i-1/2,j,k)$ and $(i+1/2,j,k)$, which are represented by `i-1,j,k` and `i,j,k`, respectively. $B^y$ and $B^z$ follow similarly.

In [11]:
%%writefile -a $Ccodesdir/compute_B_and_Bstagger_from_A.h

    // For the lower boundaries, the following applies a "copy"
    //    boundary condition on Bi and Bi_stagger where needed.
    //    E.g., Bx(imin,j,k) = Bx(imin+1,j,k)
    //    We find the copy BC works better than extrapolation.
    /******/
    /* Bx */
    /******/
    index = IDX3S(shiftedi,j,k);
    indexim1 = IDX3S(shiftedim1,j,k);
    // Set Bx = 0.5 ( Bx_stagger + Bx_stagger_im1 )
    // "Grid" Bx_stagger(i,j,k) is actually Bx_stagger(i+1/2,j,k)
    Bx[actual_index] = 0.5 * ( Bx_stagger[index] + Bx_stagger[indexim1] );

    /******/
    /* By */
    /******/
    index = IDX3S(i,shiftedj,k);
    indexjm1 = IDX3S(i,shiftedjm1,k);
    // Set By = 0.5 ( By_stagger + By_stagger_im1 )
    // "Grid" By_stagger(i,j,k) is actually By_stagger(i,j+1/2,k)
    By[actual_index] = 0.5 * ( By_stagger[index] + By_stagger[indexjm1] );

    /******/
    /* Bz */
    /******/
    index = IDX3S(i,j,shiftedk);
    indexkm1 = IDX3S(i,j,shiftedkm1);
    // Set Bz = 0.5 ( Bz_stagger + Bz_stagger_im1 )
    // "Grid" Bz_stagger(i,j,k) is actually Bz_stagger(i,j+1/2,k)
    Bz[actual_index] = 0.5 * ( Bz_stagger[index] + Bz_stagger[indexkm1] );
  }
}

Appending to GiRaFFE_standalone_Ccodes/A2B/compute_B_and_Bstagger_from_A.h


<a id='code_validation'></a>

# Step 3: Code Validation \[Back to [top](#toc)\]
$$\label{code_validation}$$

To validate the code in this tutorial we check for agreement between the files

1. that were written in this tutorial and
1. those that are generated by the python module


In [12]:
# Define the directory that we wish to validate against:
valdir = "GiRaFFE_NRPy/GiRaFFE_Ccode_library/A2B/"

import GiRaFFE_NRPy.GiRaFFE_NRPy_staggered_A2B as A2B
A2B.GiRaFFE_NRPy_A2B(valdir)

import difflib
import sys

print("Printing difference between original C code and this code...")
# Open the files to compare
files = ["compute_B_and_Bstagger_from_A.h"]

for file in files:
    print("Checking file " + file)
    with open(os.path.join(valdir,file)) as file1, open(os.path.join(Ccodesdir,file)) as file2:
        # Read the lines of each file
        file1_lines = file1.readlines()
        file2_lines = file2.readlines()
        num_diffs = 0
        for line in difflib.unified_diff(file1_lines, file2_lines, fromfile=os.path.join(valdir+file), tofile=os.path.join(Ccodesdir+file)):
            sys.stdout.writelines(line)
            num_diffs = num_diffs + 1
        if num_diffs == 0:
            print("No difference. TEST PASSED!")
        else:
            print("ERROR: Disagreement found with .py file. See differences above.")
            sys.exit(1)

Printing difference between original C code and this code...
Checking file compute_B_and_Bstagger_from_A.h
No difference. TEST PASSED!


<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_NRPy_staggered-A2B.pdf](Tutorial-GiRaFFE_NRPy_staggered-A2B.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [13]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFE_NRPy_staggered-A2B",location_of_template_file=os.path.join(".."))

Created Tutorial-GiRaFFE_NRPy_staggered-A2B.tex, and compiled LaTeX file to
    PDF file Tutorial-GiRaFFE_NRPy_staggered-A2B.pdf
